# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json
# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

City_ID            City      Lat       Lng  Max Temp  Humidity  Cloudiness  \
0        0       grytviken -54.2811  -36.5092     33.48        93          87   
1        1  puerto natales -51.7236  -72.4875     42.66        73          82   
2        2           ancud -41.8697  -73.8203     45.75        88           5   
3        3        bilibino  68.0546  166.4372      9.82        95          31   
4        4          badger  64.8000 -147.5333     25.99        84          75   

   Wind Speed Country        Date  
0       10.69      GS  1698195386  
1       13.24      CL  1698195400  
2        5.17      CL  1698195344  
3        3.18      RU  1698195400  
4        4.61      US  1698195401

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

map_plot = city_data_df.hvplot.points(
    geo=True,
    x='Lng',
    y='Lat',
    color='City',
    size='Humidity',
    tiles= 'OSM',
    alpha=0.4,
    width=800,
    height=400
)

map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [17]:
# Narrow down cities that fit criteria and drop any results with null values
vacation_criteria = ((city_data_df['Humidity'] < 70)
& (city_data_df['Max Temp'] >= 55) 
& (city_data_df['Max Temp'] <= 80)
& (city_data_df['Wind Speed'] <= 10)
& (city_data_df['Cloudiness'] <= 45)
)
cleaned_df = city_data_df[vacation_criteria]

# Drop any rows with null values
cleaned_df = cleaned_df.dropna()

# Display sample data
cleaned_df.head()


City_ID     City      Lat       Lng  Max Temp  Humidity  Cloudiness  \
7         7  dingtao  35.0683  115.5625     72.70        45           7   
15       15  copiapo -27.3667  -70.3333     55.09        50           0   
25       25  karatas  36.5820   35.3701     65.05        54           2   
37       37  dongcun  36.7767  121.1597     68.23        54           0   
56       56      leo  11.1000   -2.1000     77.45        61          40   

    Wind Speed Country        Date  
7         7.11      CN  1698195402  
15        1.34      CL  1698195404  
25        6.60      TR  1698195407  
37        3.80      CN  1698195412  
56        6.69      BF  1698195419

### Step 3: Create a new DataFrame called `hotel_df`.

In [18]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = cleaned_df.copy()
hotel_df = hotel_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

City Country      Lat       Lng  Humidity Hotel Name
7   dingtao      CN  35.0683  115.5625        45           
15  copiapo      CL -27.3667  -70.3333        50           
25  karatas      TR  36.5820   35.3701        54           
37  dongcun      CN  36.7767  121.1597        54           
56      leo      BF  11.1000   -2.1000        61

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [21]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    filters = f"circle:{longitude},{latitude},{radius}"
    bias = f"proximity:{longitude},{latitude}"
    
    params = {
    "categories": categories,
    "radius": radius,
    "lang": "en",
    "limit": 1,
    "filters": filters,
    "bias": bias,
    "apiKey": geoapify_key  
    }
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
dingtao - nearest hotel: 全季酒店
copiapo - nearest hotel: Hotel Il Riposo
karatas - nearest hotel: Kumsal Butik Otel
dongcun - nearest hotel: 烟台海阳方圆大酒店
leo - nearest hotel: Auberge le prestige
avarua - nearest hotel: Paradise Inn
manfalut - nearest hotel: University Guest House
nurdagi - nearest hotel: Nikopolis Hotel
hamilton - nearest hotel: North Vista Manor
keren - nearest hotel: Eritrea Hotel قندق إرتريا
al jawf - nearest hotel: No hotel found
taltal - nearest hotel: Hostería Taltal
sanctuary point - nearest hotel: Dungowan Accommodation
ataq - nearest hotel: الشارقة بلازا
ridgecrest - nearest hotel: Best Western China Lake Inn
ukiah - nearest hotel: Palace Hotel
tamanrasset - nearest hotel: فندق الأمان
timberlake - nearest hotel: Tru by Hilton Lynchburg
mendoza - nearest hotel: Hotel Eben-Ezer
orkney - nearest hotel: No hotel found
beipiao - nearest hotel: 燕都国际酒店
hot springs - nearest hotel: Hotel Hale
khamis mushait - nearest hotel: فندق راديانس جراند
awjilah 

City Country      Lat       Lng  Humidity  \
7              dingtao      CN  35.0683  115.5625        45   
15             copiapo      CL -27.3667  -70.3333        50   
25             karatas      TR  36.5820   35.3701        54   
37             dongcun      CN  36.7767  121.1597        54   
56                 leo      BF  11.1000   -2.1000        61   
57              avarua      CK -21.2078 -159.7750        53   
62            manfalut      EG  27.3104   30.9700        52   
92             nurdagi      TR  37.1773   36.7419        52   
138           hamilton      US  39.1834  -84.5333        64   
165              keren      ER  15.7778   38.4581        61   
179            al jawf      SA  29.5000   38.7500        59   
183             taltal      CL -25.4000  -70.4833        67   
184    sanctuary point      AU -35.1114  150.6290        52   
194               ataq      YE  14.5377   46.8319        68   
198         ridgecrest      US  35.6225 -117.6709        24   
199              ukiah      US  39.1502 -123.2078        33   
207        tamanrasset      DZ  22.7850    5.5228        18   
232         timberlake      US  37.3207  -79.2575        68   
244            mendoza      AR -32.8908  -68.8272        55   
261             orkney      ZA -26.9802   26.6727        16   
263            beipiao      CN  41.7919  120.7792        32   
291        hot springs      US  34.5037  -93.0552        45   
294     khamis mushait      SA  18.3064   42.7292        63   
305            awjilah      LY  29.1081   21.2869        32   
342             newman      US  37.3138 -121.0208        56   
369           chardara      KZ  41.2547   67.9692        62   
375           jamjamal      IQ  35.5365   44.8317        64   
383              haifa      IL  32.8156   34.9892        68   
391       todos santos      MX  23.4500 -110.2167        68   
396           hargeysa      SO   9.5600   44.0650        68   
401              ha'il      SA  27.5219   41.6907        52   
412             laguna      US  38.4210 -121.4238        55   
420              patti      IN  31.2808   74.8572        47   
432            reguiba      DZ  33.5639    6.7033        29   
452            naldurg      IN  17.8167   76.3000        58   
460              bowen      AU -20.0167  148.2333        66   
469     preobrazheniye      RU  42.9019  133.9064        60   
480            petauke      ZM -14.2426   31.3253        40   
482               ghat      LY  24.9647   10.1728        20   
488             morena      IN  26.4969   78.0000        50   
493        charlestown      AU -32.9500  151.6667        68   
495             tukrah      LY  32.5341   20.5791        52   
498            marahra      IN  27.7333   78.5833        55   
501           qaisumah      SA  28.3112   46.1273        43   
512        al kharijah      EG  25.4514   30.5464        44   
535              ta'if      SA  21.2703   40.4158        37   
539   muroto-misakicho      JP  33.2833  134.1500        63   
544  san jose del cabo      MX  23.0589 -109.6977        56   

                       Hotel Name  
7                            全季酒店  
15                Hotel Il Riposo  
25              Kumsal Butik Otel  
37                      烟台海阳方圆大酒店  
56            Auberge le prestige  
57                   Paradise Inn  
62         University Guest House  
92                Nikopolis Hotel  
138             North Vista Manor  
165     Eritrea Hotel قندق إرتريا  
179                No hotel found  
183               Hostería Taltal  
184        Dungowan Accommodation  
194                 الشارقة بلازا  
198   Best Western China Lake Inn  
199                  Palace Hotel  
207                   فندق الأمان  
232       Tru by Hilton Lynchburg  
244               Hotel Eben-Ezer  
261                No hotel found  
263                        燕都国际酒店  
291                    Hotel Hale  
294            فندق راديانس جراند  
305                No hotel found  
342          Hampton Inn & Sui

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [22]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    geo=True,
    x='Lng',
    y='Lat',
    color='City',
    size='Humidity',
    tiles= 'OSM',
    alpha=0.4,
    width=800,
    height=300,
    hover_cols=['City', 'Country', 'Hotel Name']
)


# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)